In [1]:
%pylab inline
import sqlite3
import pandas
import tqdm
from sklearn import *
import ml_metrics
from xgboost import sklearn as xgblearn

Populating the interactive namespace from numpy and matplotlib


/Users/riri/.virtualenvs/analysis3/lib/python3.4/site-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/Users/riri/.virtualenvs/analysis3/lib/python3.4/site-packages/sklearn/qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


In [6]:
admissible_cols = ['sales_depo', 'sales_channel', 'route_id', 'client_id', 'product_id']

def fit_mini_batch(reg, test=False):
    # Fit three little estimators in each
    # round of boosting, all from the same
    # sample.
    reg.n_estimators += 3
    train_batch = np.random.randint(100)
    con = sqlite3.connect('/tmp/train_test_data.sqlite3')
    try:
        print('Fetching data: %d' % train_batch)
        data = pandas.read_sql('''
            SELECT * 
              FROM train_data 
             WHERE adjusted_demand is not null 
                   AND rand = ? AND week_num >= 8''', con=con, params=[train_batch])
    finally:
        con.close()
    X = data[admissible_cols].as_matrix().copy(order='C')
    # See https://github.com/dmlc/xgboost/issues/446#issuecomment-135555130
    y = np.log(1+data.adjusted_demand.as_matrix().copy(order='C'))
    reg.fit(X, y)

    if test:
        test_batch = np.random.randint(100)
        con = sqlite3.connect('/tmp/train_test_data.sqlite3')
        try:
            data = pandas.read_sql('''
                SELECT * 
                  FROM test_data 
                 WHERE adjusted_demand is not null 
                       AND rand = ? AND week_num >= 8''', con=con, params=[test_batch])
        finally:
            con.close()
        X = data[admissible_cols].as_matrix().copy(order='C')

        pred = exp(reg.predict(X)) - 1
        print(ml_metrics.rmsle(data.adjusted_demand, np.greater(pred, 0)))
        print(ml_metrics.rmsle(data.adjusted_demand, pred))

In [7]:
reg = ensemble.GradientBoostingRegressor(
    warm_start=True,
    max_depth=11,
    verbose=0, 
    n_estimators=0)

In [8]:
for i in range(100):
    fit_mini_batch(reg, True)

Fetching data: 11
1.23548091818
0.797078233026
Fetching data: 35
1.2366507197
0.758893776333
Fetching data: 46
1.23206525135
0.725020998639
Fetching data: 66
1.23497810257
0.698710754458
Fetching data: 62
1.23833451274
0.679529854558
Fetching data: 5
1.23638488366
0.657859030445
Fetching data: 14
1.23646560663
0.641949426985
Fetching data: 49
1.23264248793
0.628687890158
Fetching data: 80


ERROR: An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 35))



KeyboardInterrupt: 